In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-347601-6e0b822fd842.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#date = 0 #当日は0、前日は1・・・
#
date = 1 #当日は0、前日は1・・・
date2 = "08/08"
pmc = 23257002 # 店舗ID
SPREADSHEET_KEY = '1btKtFirpE2xkY3g-8Iz1xABgw_TtJaVx77HTtH-qPqc' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 1057 # スロットの1台目
wait = 0.01

user_id = "g84909252+003@gmail.com"
user_pw = "sitega22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
path = "C:\\Users\\garden026\\WD\\" + str(wd) + "\\chromedriver.exe"

chrome_service = ChromeService(executable_path=path)

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\garden\\WD\\" + str(wd) + "\\chromedriver.exe"

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
#s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
#wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/08', '025579', '1', '5128.0', '-902.0', '2', '1']
['08/08', '025579', '2', '9487.0', '8427.0', '9', '2']
['08/08', '025579', '3', '8974.0', '-5115.0', '1', '1']
['08/08', '025579', '5', '16923.0', '-7522.0', '3', '2']
['08/08', '025579', '6', '9487.0', '302.0', '4', '2']
['08/08', '025579', '7', '4872.0', '-3911.0', '0', '0']
['08/08', '025579', '8', '4103.0', '-902.0', '2', '2']
['08/08', '025579', '10', '5897.0', '-2707.0', '1', '1']
['08/08', '025579', '11', '6410.0', '-1504.0', '2', '1']
['08/08', '025579', '12', '4615.0', '-3610.0', '0', '0']
['08/08', '025765', '13', '14103.0', '-902.0', '14', '8']
['08/08', '025765', '15', '4359.0', '603.0', '6', '3']
['08/08', '025765', '16', '0', '0', '1', '1']
['08/08', '025765', '17', '22821.0', '2408.0', '30', '9']
['08/08', '025765', '18', '4103.0', '3311.0', '11', '3']
['08/08', '025605', '20', '3590.0', '-1504.0', '2', '1']
['08/08', '025605', '21', '2821.0', '603.0', '5', '1']
['08/08', '025605', '22', '9231.0', '-902.0', '9', '4']

['08/08', '025869', '217', '55128.0', '-23471.0', '25', '7']
['08/08', '025869', '218', '52308.0', '-6018.0', '39', '10']
['08/08', '025869', '220', '44615.0', '-24675.0', '13', '4']
['08/08', '025869', '221', '41795.0', '-29490.0', '6', '2']
['08/08', '025869', '222', '46667.0', '-23772.0', '16', '6']
['08/08', '025869', '223', '46154.0', '10232.0', '48', '8']
['08/08', '025825', '225', '22308.0', '8427.0', '18', '18']
['08/08', '025825', '226', '11795.0', '3311.0', '9', '9']
['08/08', '025825', '227', '14103.0', '7223.0', '13', '13']
['08/08', '025825', '228', '32308.0', '-17754.0', '7', '7']
['08/08', '025825', '230', '20256.0', '-11434.0', '4', '4']
['08/08', '025761', '231', '15641.0', '-8726.0', '14', '4']
['08/08', '025761', '232', '6923.0', '-5416.0', '3', '2']
['08/08', '025606', '233', '17949.0', '-3309.0', '8', '3']
['08/08', '025606', '235', '18718.0', '-10832.0', '3', '1']
['08/08', '025606', '236', '14615.0', '-10832.0', '1', '1']
['08/08', '025606', '237', '25385.0', '-1

['08/08', '025544', '531', '8205.0', '1205.0', '29', '2']
['08/08', '025637', '532', '11282.0', '-5717.0', '5', '4']
['08/08', '025523', '533', '5385.0', '2408.0', '11', '3']
['08/08', '025722', '535', '1538.0', '-1203.0', '0', '0']
['08/08', '025865', '536', '9487.0', '-3610.0', '13', '2']
['08/08', '025865', '537', '4872.0', '-902.0', '13', '3']
['08/08', '025865', '538', '2051.0', '-1805.0', '1', '1']
['08/08', '025865', '550', '2051.0', '-902.0', '5', '2']
['08/08', '025865', '551', '5385.0', '2408.0', '24', '3']
['08/08', '025609', '552', '0', '0', '0', '0']
['08/08', '025609', '553', '0', '0', '0', '0']
['08/08', '025835', '555', '12821.0', '-1203.0', '42', '4']
['08/08', '025835', '556', '6154.0', '-2707.0', '12', '2']
['08/08', '025891', '557', '24359.0', '-1504.0', '41', '24']
['08/08', '025891', '558', '30000.0', '-13240.0', '27', '18']
['08/08', '025859', '560', '4615.0', '-2707.0', '1', '1']
['08/08', '025859', '561', '5385.0', '22269.0', '28', '28']
['08/08', '025876', '56

['08/08', '025522', '766', '513.0', '-902.0', '0', '0']
['08/08', '025840', '767', '1538.0', '-1504.0', '0', '0']
['08/08', '025782', '768', '4872.0', '9330.0', '23', '2']
['08/08', '025801', '770', '0', '0', '0', '0']
['08/08', '025712', '771', '4615.0', '-902.0', '11', '3']
['08/08', '025656', '772', '1282.0', '-1203.0', '0', '0']
['08/08', '025690', '773', '769.0', '-1203.0', '1', '1']
['08/08', '025328', '775', '17179.0', '11135.0', '22', '2']
['08/08', '025550', '776', '7179.0', '9931.0', '9', '1']
['08/08', '025599', '777', '5641.0', '-4513.0', '1', '1']
['08/08', '025667', '778', '6410.0', '-5416.0', '0', '0']
['08/08', '025747', '780', '2051.0', '302.0', '4', '2']
['08/08', '025873', '781', '24359.0', '-20161.0', '0', '0']
['08/08', '025328', '782', '0', '0', '0', '0']
['08/08', '025595', '783', '9231.0', '-3309.0', '6', '2']
['08/08', '025787', '785', '7436.0', '1806.0', '59', '4']
['08/08', '025873', '786', '0', '0', '0', '0']
['08/08', '025701', '787', '3333.0', '-2707.0', '

['08/08', '120122', '1082', '7268', '3333.0', '94', '27', '--']
['08/08', '120122', '1083', '6633', '3102.0', '90', '27', '--']
['08/08', '120122', '1085', '3465', '833.0', '40', '15', '--']
['08/08', '120122', '1086', '4989', '-1667.0', '41', '11', '--']
['08/08', '120122', '1087', '4492', '-185.0', '45', '16', '--']
['08/08', '120122', '1088', '5503', '4676.0', '91', '14', '--']
['08/08', '120122', '1100', '5451', '-1528.0', '51', '12', '--']
['08/08', '120103', '1101', '412', '139.0', '4', '0', '1']
['08/08', '120103', '1102', '2191', '-93.0', '16', '2', '8']
['08/08', '120103', '1103', '1148', '-648.0', '3', '0', '2']
['08/08', '120103', '1105', '2158', '648.0', '25', '1', '8']
['08/08', '120103', '1106', '1409', '2593.0', '31', '0', '2']
['08/08', '120103', '1107', '4143', '2824.0', '37', '1', '10']
['08/08', '120130', '1108', '5209', '6019.0', '0', '7', '31']
['08/08', '120130', '1110', '2985', '602.0', '0', '4', '13']
['08/08', '120130', '1111', '2128', '-417.0', '0', '4', '10']

['08/08', '120150', '1286', '9594', '93.0', '0', '34', '111']
['08/08', '120150', '1287', '8621', '1204.0', '0', '34', '110']
['08/08', '120110', '1288', '4936', '-3889.0', '16', '6', '--']
['08/08', '120110', '1300', '3652', '880.0', '28', '8', '--']
['08/08', '120110', '1301', '3851', '-3889.0', '7', '8', '--']
['08/08', '120110', '1302', '2269', '1389.0', '22', '4', '--']
['08/08', '120110', '1303', '4820', '-972.0', '24', '17', '--']
['08/08', '120110', '1305', '3450', '-2083.0', '9', '13', '--']
['08/08', '120110', '1306', '1944', '-833.0', '9', '3', '--']
['08/08', '120110', '1307', '1990', '-93.0', '12', '8', '--']
['08/08', '120110', '1308', '2660', '-417.0', '14', '8', '--']
['08/08', '120110', '1310', '4137', '880.0', '30', '11', '--']
['08/08', '120110', '1311', '3935', '-3102.0', '12', '6', '--']
['08/08', '120110', '1312', '2571', '-2037.0', '8', '6', '--']
['08/08', '120110', '1313', '2790', '-3009.0', '4', '5', '--']
['08/08', '120110', '1315', '4132', '-2546.0', '14', '

['08/08', '120010', '1606', '1559', '-93.0', '6', '3', '--']
['08/08', '120010', '1607', '4531', '1852.0', '26', '10', '--']
['08/08', '120010', '1608', '407', '-231.0', '1', '1', '--']
['08/08', '120010', '1610', '8240', '2315.0', '40', '26', '--']
['08/08', '120010', '1611', '438', '-185.0', '1', '1', '--']
['08/08', '120010', '1612', '3776', '-972.0', '12', '8', '--']
['08/08', '120010', '1613', '3921', '231.0', '13', '14', '--']
['08/08', '120010', '1615', '1357', '139.0', '5', '4', '--']
['08/08', '120010', '1616', '1037', '231.0', '5', '2', '--']
['08/08', '120010', '1617', '6320', '-1944.0', '17', '14', '--']
['08/08', '120010', '1618', '1417', '-417.0', '3', '3', '--']
['08/08', '120010', '1620', '1888', '-556.0', '5', '6', '--']
['08/08', '120010', '1621', '5505', '-694.0', '18', '18', '--']
['08/08', '119916', '1622', '3725', '185.0', '16', '7', '--']
['08/08', '119916', '1623', '359', '-93.0', '1', '0', '--']
['08/08', '119916', '1625', '2947', '972.0', '13', '9', '--']
['08

{'spreadsheetId': '1btKtFirpE2xkY3g-8Iz1xABgw_TtJaVx77HTtH-qPqc',
 'updates': {'spreadsheetId': '1btKtFirpE2xkY3g-8Iz1xABgw_TtJaVx77HTtH-qPqc',
  'updatedRange': 'S20RAW!F70672:M71072',
  'updatedRows': 401,
  'updatedColumns': 8,
  'updatedCells': 3208}}

In [18]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
#wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/04', '025579', '1', '2564.0', '-2106.0', '0', '0']
['08/04', '025579', '2', '7436.0', '12339.0', '10', '2']
['08/04', '025579', '3', '24103.0', '9931.0', '17', '2']
['08/04', '025579', '5', '2308.0', '603.0', '2', '1']
['08/04', '025579', '6', '4359.0', '302.0', '3', '1']
['08/04', '025579', '7', '2051.0', '-1504.0', '0', '0']
['08/04', '025579', '8', '4103.0', '-3309.0', '0', '0']
['08/04', '025579', '10', '10513.0', '-902.0', '5', '2']
['08/04', '025579', '11', '21282.0', '-10832.0', '3', '2']
['08/04', '025579', '12', '513.0', '-902.0', '0', '0']
['08/04', '025765', '13', '13333.0', '-2707.0', '7', '3']
['08/04', '025765', '15', '1795.0', '-902.0', '3', '2']
['08/04', '025765', '16', '6667.0', '-3911.0', '2', '2']
['08/04', '025765', '17', '4359.0', '2107.0', '9', '4']
['08/04', '025765', '18', '5897.0', '3010.0', '13', '3']
['08/04', '025605', '20', '1538.0', '-1504.0', '0', '0']
['08/04', '025605', '21', '1282.0', '-902.0', '1', '1']
['08/04', '025605', '22', '3077.0', '-2707

['08/04', '025709', '218', '8718.0', '-6319.0', '2', '2']
['08/04', '025606', '220', '14103.0', '-10231.0', '1', '1']
['08/04', '025606', '221', '12821.0', '15950.0', '17', '6']
['08/04', '025606', '222', '5897.0', '-1504.0', '2', '1']
['08/04', '025606', '223', '24872.0', '904.0', '15', '3']
['08/04', '025825', '225', '23333.0', '-9629.0', '6', '6']
['08/04', '025825', '226', '23590.0', '-14744.0', '4', '4']
['08/04', '025825', '227', '15641.0', '9631.0', '15', '15']
['08/04', '025825', '228', '19231.0', '-8726.0', '5', '5']
['08/04', '025825', '230', '22051.0', '302.0', '13', '13']
['08/04', '025761', '231', '8462.0', '3612.0', '26', '3']
['08/04', '025761', '232', '7692.0', '4214.0', '32', '4']
['08/04', '025802', '233', '0', '0', '0', '0']
['08/04', '025802', '235', '8974.0', '-6018.0', '7', '7']
['08/04', '025767', '236', '2308.0', '3311.0', '8', '2']
['08/04', '025661', '237', '16923.0', '-14143.0', '1', '1']
['08/04', '025732', '238', '2051.0', '-2106.0', '0', '0']
['08/04', '02

['08/04', '025523', '533', '5897.0', '-902.0', '6', '2']
['08/04', '025722', '535', '6410.0', '1806.0', '13', '5']
['08/04', '025865', '536', '19487.0', '-2406.0', '35', '11']
['08/04', '025865', '537', '1026.0', '-902.0', '1', '1']
['08/04', '025865', '538', '8718.0', '-4513.0', '10', '4']
['08/04', '025865', '550', '1795.0', '2709.0', '13', '2']
['08/04', '025865', '551', '7692.0', '3010.0', '34', '4']
['08/04', '025875', '552', '11026.0', '6320.0', '18', '3']
['08/04', '025875', '553', '7436.0', '-6018.0', '2', '2']
['08/04', '025875', '555', '0', '0', '0', '0']
['08/04', '025835', '556', '2051.0', '6922.0', '30', '1']
['08/04', '025835', '557', '6923.0', '-2106.0', '16', '3']
['08/04', '025835', '558', '15385.0', '-1504.0', '45', '5']
['08/04', '025859', '560', '3846.0', '11135.0', '17', '17']
['08/04', '025859', '561', '0', '0', '0', '0']
['08/04', '025876', '562', '5385.0', '7825.0', '11', '11']
['08/04', '025876', '563', '5128.0', '11737.0', '14', '14']
['08/04', '025645', '565'

['08/04', '025782', '768', '6410.0', '-902.0', '8', '1']
['08/04', '025801', '770', '5897.0', '-4814.0', '0', '0']
['08/04', '025712', '771', '7692.0', '2107.0', '26', '5']
['08/04', '025656', '772', '0', '0', '0', '0']
['08/04', '025690', '773', '5897.0', '-4814.0', '0', '0']
['08/04', '025328', '775', '5128.0', '2709.0', '7', '2']
['08/04', '025550', '776', '6923.0', '-5416.0', '0', '0']
['08/04', '025743', '777', '6154.0', '-5115.0', '1', '1']
['08/04', '025667', '778', '3077.0', '-2707.0', '0', '0']
['08/04', '025747', '780', '5897.0', '-5115.0', '0', '0']
['08/04', '025873', '781', '4103.0', '14144.0', '12', '12']
['08/04', '025328', '782', '0', '0', '0', '0']
['08/04', '025595', '783', '8974.0', '-4513.0', '5', '1']
['08/04', '025787', '785', '1282.0', '-1203.0', '4', '1']
['08/04', '025873', '786', '16410.0', '-14143.0', '0', '0']
['08/04', '025701', '787', '2564.0', '11135.0', '23', '1']
['08/04', '025667', '788', '8462.0', '4515.0', '7', '1']
['08/04', '025703', '800', '6923.0

['08/04', '120103', '1087', '1567', '-833.0', '3', '0', '4']
['08/04', '120103', '1088', '894', '-602.0', '2', '0', '3']
['08/04', '120103', '1100', '1538', '-787.0', '6', '3', '5']
['08/04', '120103', '1101', '0', '0', '0', '0', '--']
['08/04', '120103', '1102', '0', '0', '0', '0', '--']
['08/04', '120103', '1103', '0', '0', '0', '0', '--']
['08/04', '120103', '1105', '514', '787.0', '8', '0', '3']
['08/04', '120103', '1106', '0', '0', '0', '0', '--']
['08/04', '120103', '1107', '0', '0', '0', '0', '--']
['08/04', '120130', '1108', '4958', '-1713.0', '0', '8', '20']
['08/04', '120130', '1110', '3427', '2176.0', '0', '9', '27']
['08/04', '120130', '1111', '3719', '2222.0', '0', '4', '21']
['08/04', '120130', '1112', '2380', '-602.0', '0', '6', '13']
['08/04', '120130', '1113', '4833', '-1852.0', '0', '9', '19']
['08/04', '120104', '1115', '514', '-463.0', '0', '0', '1']
['08/04', '120104', '1116', '275', '-93.0', '0', '0', '1']
['08/04', '120145', '1117', '4256', '1389.0', '0', '5', '4

['08/04', '120110', '1305', '3218', '-93.0', '19', '12', '--']
['08/04', '120110', '1306', '3676', '1481.0', '27', '17', '--']
['08/04', '120110', '1307', '2958', '-139.0', '19', '6', '--']
['08/04', '120110', '1308', '4007', '-2083.0', '15', '9', '--']
['08/04', '120110', '1310', '2615', '2824.0', '30', '7', '--']
['08/04', '120110', '1311', '3246', '-2269.0', '11', '7', '--']
['08/04', '120110', '1312', '2232', '370.0', '13', '12', '--']
['08/04', '120110', '1313', '2259', '1806.0', '23', '7', '--']
['08/04', '120110', '1315', '6317', '-4907.0', '15', '12', '--']
['08/04', '120110', '1316', '2314', '1620.0', '20', '9', '--']
['08/04', '120110', '1317', '4104', '2269.0', '33', '18', '--']
['08/04', '120110', '1318', '5049', '-3333.0', '18', '10', '--']
['08/04', '120110', '1320', '4516', '-2870.0', '18', '5', '--']
['08/04', '120110', '1321', '1992', '-880.0', '10', '3', '--']
['08/04', '120110', '1322', '5656', '-4769.0', '15', '8', '--']
['08/04', '120110', '1323', '3817', '-2685.0'

['08/04', '120010', '1613', '1756', '-926.0', '3', '4', '--']
['08/04', '120010', '1615', '3986', '-694.0', '11', '13', '--']
['08/04', '120010', '1616', '6057', '1713.0', '29', '19', '--']
['08/04', '120010', '1617', '1090', '-185.0', '4', '2', '--']
['08/04', '120010', '1618', '1266', '-741.0', '2', '2', '--']
['08/04', '120010', '1620', '5128', '694.0', '25', '12', '--']
['08/04', '120010', '1621', '1438', '-324.0', '5', '2', '--']
['08/04', '119916', '1622', '568', '-741.0', '0', '0', '--']
['08/04', '119916', '1623', '822', '-139.0', '3', '1', '--']
['08/04', '119916', '1625', '512', '-231.0', '1', '1', '--']
['08/04', '119916', '1626', '614', '-417.0', '1', '0', '--']
['08/04', '119916', '1627', '2626', '1019.0', '15', '5', '--']
['08/04', '119916', '1628', '341', '-93.0', '1', '1', '--']
['08/04', '119916', '1630', '746', '324.0', '3', '2', '--']
['08/04', '119916', '1631', '1598', '-93.0', '7', '4', '--']
['08/04', '119916', '1632', '1035', '-139.0', '3', '1', '--']
['08/04', '

{'spreadsheetId': '1btKtFirpE2xkY3g-8Iz1xABgw_TtJaVx77HTtH-qPqc',
 'updates': {'spreadsheetId': '1btKtFirpE2xkY3g-8Iz1xABgw_TtJaVx77HTtH-qPqc',
  'updatedRange': 'S20RAW!F69526:M69907',
  'updatedRows': 382,
  'updatedColumns': 8,
  'updatedCells': 3056}}